## Example of querying a set of documents with sources using the Hugging Face Text Generation Inference server with Llama2, Langchain and a custom prompt

### Set the Inference server url (replace with your own address)

In [1]:
inference_server_url = "http://hf-tgi.llm-hosting.svc.cluster.local:3000/"

### Load some data from the folder where we have stored the PDF documentation

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader

pdf_folder_path = 'rhods-doc'

loader = PyPDFDirectoryLoader(pdf_folder_path)
docs = loader.load()

### Split the data in chunks large enough to have meaningful answers, and some overlap not to miss anything

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1024, chunk_overlap = 40)
all_splits = text_splitter.split_documents(docs)

### Store the data as embeddings in a vector database (Chroma)

In [4]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=HuggingFaceEmbeddings())

Unable to connect optimized C data functions [No module named '_testbuffer'], falling back to pure Python


### Test data retrieving

In [5]:
question = "How do I create a Data Science Project"
docs = vectorstore.similarity_search(question)
docs

[Document(page_content='-\n, and must start and end with an alphanumeric\ncharacter.\n5\n. \nEnter a \ndescription\n for your data science project.\n6\n. \nClick \nCreate\n.\nA project details page opens. From here, you can create workbenches, add cluster storage, and\nadd data connections to your project.\nVerification\nThe data science project that you created is displayed on the \nData science projects\n page.\nCHAPTER 4. CREATING A DATA SCIENCE PROJECT\n9', metadata={'source': 'rhods-doc/red_hat_openshift_data_science_self-managed-1.32-getting_started_with_red_hat_openshift_data_science_self-managed-en-us.pdf', 'page': 12}),
 Document(page_content='-\n, and must start and end with an alphanumeric\ncharacter.\n5\n. \nEnter a \ndescription\n for your data science project.\n6\n. \nClick \nCreate\n.\nA project details page opens. From here, you can create workbenches, add cluster storage, and\nCHAPTER 3. WORKING ON DATA SCIENCE PROJECTS\n11', metadata={'source': 'rhods-doc/red_hat_open

### Create the chain

In [6]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceTextGenInference
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

# NOTE: This template syntax is specific to Llama2
template="""<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant.
You will be given a question you need to answer, and a context to provide you with information. You must answer the question based as much as possible on this context.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

Question: {question}
Context: {context} [/INST]
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm = HuggingFaceTextGenInference(
    inference_server_url=inference_server_url,
    max_new_tokens=512,
    top_k=10,
    top_p=0.95,
    typical_p=0.95,
    temperature=0.1,
    repetition_penalty=1.175,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectorstore.as_retriever(),
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=True)

### Launch the query

In [7]:
question = "How do I create a Data Science Project?"
result = qa_chain({"query": question})

 Sure! I'd be happy to assist you with creating a data science project. Based on the provided context, here are the steps to follow:

1. From the OpenShift Data Science dashboard, click on "Data Science Projects" .
2. Click on "Create data science project" .
3. Enter a name for your data science project.
4. Optional: Edit the resource name for your data science project. The resource name must consist of lowercase alphanumeric characters, hyphens (-), and underscores (_).

Please note that the resource name cannot begin with a number or contain spaces. It's important to choose a descriptive name for your project that reflects its purpose and contents.

Once you have entered the required information, click "Create" to create your data science project. This will open a project details page where you can add workbenches, cluster storage, data connections, and model servers to enhance the capabilities of your project.

I hope this helps! Let me know if you have any further questions or conc

### Print the sources

In [8]:
def remove_duplicates(input_list):
    unique_list = []
    for item in input_list:
        if item.metadata['source'] not in unique_list:
            unique_list.append(item.metadata['source'])
    return unique_list

results = remove_duplicates(result['source_documents'])

for s in results:
    print(s)

rhods-doc/red_hat_openshift_data_science_self-managed-1.32-getting_started_with_red_hat_openshift_data_science_self-managed-en-us.pdf
rhods-doc/red_hat_openshift_data_science_self-managed-1.32-working_on_data_science_projects-en-us.pdf
